<a href="https://colab.research.google.com/github/matz9x/gcloud-object-detection/blob/master/odapi_gcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Cloud object detection instance

### License

**The MIT License**

Copyright (c) 2018  Mattia Varile

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### Google Cloud Bucket structure

Google bucket internal structure should respect the following scheme:

- {*name*}-detector-bucket
    - model-{*net-model*}{*version*} 
        - data
            - model.ckpt.index
            - model.ckpt.meta
            - model.ckpt.data-00000-of-00001
            - pipeline.config
            - label_map.pbtxt
            - test.record
            - train.record
        - train
        
*name*: one-word to the define the project (e.g.: cloud, ship, house, ...) 

*net-model*: short neural network descriptor (e.g.: ssdmobilev2, ssdinceptionv3, yolo3, ...)

*version*: model version (integer number)

## Gcloud authentication

Authenticate yourself in order to access Google Cloud resources.

### Google cloud authentication

In [0]:
!gcloud auth login

### Setting project in gcloud

Setting the project name to use

In [0]:
!gcloud config set project ship-detector-1

## Initialization

In [0]:
# Installing ODAPI and setting up things

%cd
!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive
!pip install -q pycocotools
!pip install -q gcloud

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

##
!python setup.py install
!cp -r object_detection ../../../usr/local/lib/python3.6/dist-packages
!cp -r slim/nets ../../../usr/local/lib/python3.6/dist-packages
!python object_detection/builders/model_builder_test.py

%cd ../../../content
!cp -r ~/models/research/object_detection /content
##

%cd ../root/models/research
!bash object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
!python setup.py sdist
!(cd slim && python setup.py sdist)

%cd ../../..
!mkdir content/gpack
!cp /tmp/pycocotools/pycocotools-2.0.tar.gz /content/gpack
!cp root/models/research/slim/dist/slim-0.1.tar.gz /content/gpack
!cp root/models/research/dist/object_detection-0.1.tar.gz /content/gpack

## Importing base model

This cell will import baseline model from tensorflow/research website. This model will be used as initial status for the network for transfer learning.

In the next cell please the baseline model name to use (find more: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)

Once required, load *train.record*, *test.record* and *label_map.pbtxt* (loaded files should have the previously specified conventional names)

In [0]:
### INPUT START

BASE_MODEL = 'ssdlite_mobilenet_v2_coco_2018_05_09'
BUCKET_NAME = 'ship-detector-bucket'
MODEL_NAME = 'model-ssdlitemobilev2-1'

### INPUT END

# Upload tfrecords
%cd ../../..
%cd content/gpack

from google.colab import files

uploaded = files.upload()

print('Please upload: train.record | test.record | label_map.pbtxt')
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

In [0]:
MODEL_FILE = BASE_MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'baseline_model'

import requests
import tarfile
import os
import shutil
import re

data_path  = os.path.join('gs://', BUCKET_NAME, MODEL_NAME, 'data')
train_path = os.path.join('gs://', BUCKET_NAME, MODEL_NAME, 'train')

# Downloading base model from tensorflow website
url = DOWNLOAD_BASE + MODEL_FILE
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

# Extracting base model
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# Cleaning up
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(BASE_MODEL, DEST_DIR)

# Edit model config file
filename = os.path.join(DEST_DIR, 'pipeline.config')
with open(filename) as f:
    s = f.read()
with open(filename, 'w') as f:
    s = re.sub('num_classes: 90', 'num_classes: 1', s)
    s = re.sub('height: 300', 'height: 600', s)
    s = re.sub('width: 300' , 'width: 600' , s)
    s = re.sub('batch_size: 24' , 'batch_size: 16' , s)
    s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', 
               os.path.join(data_path, 'model.ckpt'), s)
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record', 
               os.path.join(data_path, 'train.tfrecord'), s)
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record', 
               os.path.join(data_path, 'test.tfrecord'), s)
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', 
               os.path.join(data_path, 'label_map.pbtxt'), s)
    f.write(s)

os.environ["data_path"]  = data_path
os.environ["train_path"] = train_path

# Uploading file to Gcloud
!gsutil cp "baseline_model/model.ckpt.*" ${"data_path"}
!gsutil cp "baseline_model/pipeline.config" ${"data_path"}
!gsutil cp "train.tfrecord" ${"data_path"}
!gsutil cp "test.tfrecord" ${"data_path"}
!gsutil cp "label_map.pbtxt" ${"data_path"}

# Define model variables to send to training command
MODEL_DIR = os.path.join(BUCKET_NAME, MODEL_NAME, 'train')
PIPELINE_CONFIG_PATH = os.path.join(BUCKET_NAME, MODEL_NAME, 'data/pipeline.config')
os.environ["MODEL_DIR"] = MODEL_DIR
os.environ["PIPELINE_CONFIG_PATH"] = PIPELINE_CONFIG_PATH

## YAML setting

YAML file is required by Google Cloud services in order to specify which hardware configuration to use (https://cloud.google.com/ml-engine/docs/tensorflow/training-jobs, https://cloud.google.com/ml-engine/docs/tensorflow/using-gpus)

In [0]:
### INPUT START

s = """trainingInput:
    runtimeVersion: "1.9"
    pythonVersion: "2.7"
    scaleTier: CUSTOM
    masterType: standard_gpu
    workerCount: 1
    workerType: standard_gpu
    parameterServerCount: 1
    parameterServerType: standard"""

### INPUT END

print(s)

f = open("config.yaml", "w")
f.write(s)
f.close()

%cd ../../..

1. A job with n workers will have n + 1 training machines (n workers + 1 master).
2. The number of parameters servers used should be an odd number to prevent a parameter server from storing only weight variables or only bias variables (due to round robin parameter scheduling).
3. The learning rate in the training config should be decreased when using a larger number of workers. Some experimentation is required to find the optimal learning rate.

## Training initialization

The following command starts the training procedure. A positive execution will display:


```
state: QUEUED
```



In [0]:
%%bash
PATH_TO_LOCAL_YAML_FILE=config.yaml
gcloud ml-engine jobs submit training object_detection_`date +%m_%d_%Y_%H_%M_%S` \
    --runtime-version 1.9 \
    --job-dir=gs://${MODEL_DIR} \
    --packages object_detection-0.1.tar.gz,slim-0.1.tar.gz,pycocotools-2.0.tar.gz \
    --module-name object_detection.model_main \
    --region europe-west1 \
    --config ${PATH_TO_LOCAL_YAML_FILE} \
    -- \
    --model_dir=gs://${MODEL_DIR} \
    --pipeline_config_path=gs://${PIPELINE_CONFIG_PATH}

Start tensorboard from terminal by executing the following command from terminal


In [8]:
print("tensorboard --logdir=gs://" + BUCKET_NAME + '/' + MODEL_NAME)

tensorboard --logdir=gs://ship-detector-bucket/model-ssdlitemobilev2-1


In [0]:
# Interrupting job execution (PLEASE BE CAREFUL)
!gcloud ml-engine jobs cancel object_detection_10_25_2018_11_36_10

In [0]:
# start streaming log by defining job name
!gcloud ml-engine jobs stream-logs object_detection_10_25_2018_08_42_46

## Exporting TF lite model (under development)

In [0]:
# Installing bazel
!apt-get install openjdk-8-jdk
!echo "deb [arch=amd64] http://storage.googleapis.com/bazel-apt stable jdk1.8" | tee /etc/apt/sources.list.d/bazel.list
!curl https://bazel.build/bazel-release.pub.gpg | apt-key add -
!apt-get update && apt-get install bazel
!apt-get upgrade bazel

In [0]:
%%bash
export CONFIG_FILE=$PIPELINE_CONFIG_PATH
export CHECKPOINT_PATH=$MODEL_DIR
export OUTPUT_DIR=/tflite

In [0]:
import os
%cd ../../..

BUCKET_NAME = 'ship-detector-bucket'
MODEL_NAME = 'model-ssdlitemobilev2-1'
MODEL_DIR = os.path.join(BUCKET_NAME, MODEL_NAME, 'train')

In [0]:
# Download objects from google cloud

%%bash
mkdir content/gcloudfiles
mkdir content/outputfiles
gsutil cp gs://${MODEL_DIR}/model.* content/gcloudfiles
gsutil cp gs://${MODEL_DIR}/pipeline.config content/gcloudfiles
gsutil cp gs://${MODEL_DIR}/graph.pbtxt content/gcloudfiles
export CHECKPOINT_PATH=content/gcloudfiles/model.ckpt-5382
export CONFIG_FILE=content/gcloudfiles/pipeline.config
export OUTPUT_DIR=content/outputfiles

python content/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$CONFIG_FILE \
--trained_checkpoint_prefix=$CHECKPOINT_PATH \
--output_directory=$OUTPUT_DIR \
--add_postprocessing_op=true

In [0]:
# TOCO Converting process
%%bash
%cd ../../..
touch WORKSPACE
bazel run -c opt //usr/local/lib/python3.6/dist-packages/tensorflow/contrib/lite/toco:toco -- \
--input_file=$OUTPUT_DIR/tflite_graph.pb \
--output_file=$OUTPUT_DIR/detect.tflite \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

In [0]:
%%bash
python content/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$CONFIG_FILE \
--trained_checkpoint_prefix=$CHECKPOINT_PATH \
--output_directory=$OUTPUT_DIR \
--add_postprocessing_op=true